Tensor RTに変換された学習済みモデルをつかって自動走行します。

In [1]:
import torch
import torchvision

CATEGORIES = ['apex']

device = torch.device('cuda')
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2 * len(CATEGORIES))
model = model.cuda().eval().half()

Tensor RT形式のモデルを読み込む。

In [2]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('road_following_model_trt.pth'))

racecarクラスをインスタンス化する

In [3]:
from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar()

96


カメラの起動のためにカメラを制御するnvargus-daemonの再起動。

In [4]:
!echo jetson | sudo -S systemctl restart nvargus-daemon

[sudo] password for jetson: 

カメラクラスをインスタンス化する。

In [5]:
from jetcam.csi_camera import CSICamera

camera = CSICamera(width=224, height=224, capture_fps=65)

最後に、JetRacerを下に置き、下記セルを実行します。

* 車の左右にうまく回らない場合は、`STEERING_GAIN` を小さいくする
* ターンがうまくいかない場合は、`STEERING_GAIN`を大きくする
* 車が左に傾く場合は、`STEERING_BIAS`を値を-0.05ぐらいづつマイナスにする
* 車が右に傾く場合は、`STEERING_BIAS`を値を+0.05ぐらいづつプラスにする

In [ ]:
from utils import preprocess
import numpy as np

STEERING_GAIN = 1.1
STEERING_BIAS = 0.00

car.throttle = 0.46

while True:
    image = camera.read()
    image = preprocess(image).half()
    output = model_trt(image).detach().cpu().numpy().flatten()
    x = -float(output[0])
    car.steering = x * STEERING_GAIN + STEERING_BIAS